In [9]:
from typing import List
import random
import numpy as np

import torch; print(torch.__version__)
from torch import nn, autograd, Tensor
from torch.nn import functional as F

from src.models import PinnBaseline
from src.dataset import get_orig_dataset
from src.trainer import Trainer

2.1.2+cu121


In [3]:
torch.random.manual_seed(0)
random.seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [4]:
# Model
hidden_dims = [20] * 8
model = PinnBaseline(hidden_dims=hidden_dims)
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {num_params}")

# Data
data_path="C:/Users/enomazosii/OneDrive/document/GitHub/piml/firstProject/data/cylinder_nektar_wake.mat"
train_data, test_data, min_x, max_x = get_orig_dataset(data_path)

Number of parameters: 3064


In [5]:
trainer = Trainer(model,batch_size=500000)
trainer.train(train_data)

Output dir: result\pinnBaseline\bs500000-lr0.001-lrstep5-lrgamma0.8-epoch100


====== Training ======
device is "cuda"
# epochs: 100
# examples: 1000000
# samples used per epoch: 5000
batch size: 500000
# steps: 1
====== Epoch 0 ======
{'step': 0, 'loss': 4237.025879, 'lr': 0.001, 'lambda1': -0.001, 'lambda2': -0.001, 'u_loss': 3841.153809, 'v_loss': 376.606995, 'f_u_loss': 15.493012, 'f_v_loss': 3.771802, 'time': 106.5}
Checkpointing to result\pinnBaseline\bs500000-lr0.001-lrstep5-lrgamma0.8-epoch100\ckpt-0
====== Epoch 0 done ======
====== Epoch 1 ======
{'step': 0, 'loss': 4167.817871, 'lr': 0.0008, 'lambda1': -0.001, 'lambda2': -0.0017, 'u_loss': 3779.161133, 'v_loss': 369.927338, 'f_u_loss': 14.390205, 'f_v_loss': 4.339289, 'time': 213.4}
Checkpointing to result\pinnBaseline\bs500000-lr0.001-lrstep5-lrgamma0.8-epoch100\ckpt-1
====== Epoch 1 done ======
====== Epoch 2 ======
{'step': 0, 'loss': 4176.466797, 'lr': 0.0006, 'lambda1': -0.0006, 'lambda2': -0.0023, 'u_loss': 3776.88208, 'v_loss': 379.266876, 'f_u_loss': 15.433453, 'f_v_loss': 4.884097, 'time': 319

In [6]:
lambda1 = trainer.model.lambda1.item()
lambda2 = trainer.model.lambda2.item()
print(lambda1, lambda2)

0.0017006738344207406 -0.004586071241647005


In [7]:
outputs = trainer.predict(test_data)
preds = outputs['preds']
loss = outputs['loss']
preds = preds.detach().cpu().numpy()
print("loss:", loss)
print("preds:")
print(preds)

====== Testing ======
# examples: 1000
batch size: 16000000
# steps: 1
====== Testing done ======
loss: 751.6759033203125
preds:
[[-0.07060594  0.01479189  0.01930717]
 [ 0.5592356   0.37817597  0.05417094]
 [ 0.6146568   0.21368054  0.03177109]
 ...
 [-0.05995093  0.0289647   0.01586445]
 [ 0.04307128  0.15295637 -0.01227666]
 [ 0.58110183  0.10916795  0.03541075]]


In [10]:
test_arr = np.array(test_data.data)
p = test_arr[:, 3]
u = test_arr[:, 4]
v = test_arr[:, 5]
p_pred = preds[:, 0]
u_pred = preds[:, 1]
v_pred = preds[:, 2]

# Error
err_u = np.linalg.norm(u - u_pred, 2) / np.linalg.norm(u, 2)
err_v = np.linalg.norm(v - v_pred, 2) / np.linalg.norm(v, 2)
err_p = np.linalg.norm(p - p_pred, 2) / np.linalg.norm(p, 2)

err_lambda1 = np.abs(lambda1 - 1.0)
err_lambda2 = np.abs(lambda2 - 0.01) / 0.01

print(f"Error in velocity: {err_u:.2e}, {err_v:.2e}")
print(f"Error in pressure: {err_p:.2e}")
print(f"Error in lambda 1: {err_lambda1:.2f}")
print(f"Error in lambda 2: {err_lambda2:.2f}")

Error in velocity: 9.10e-01, 9.89e-01
Error in pressure: 2.67e+00
Error in lambda 1: 1.00
Error in lambda 2: 1.46
